<a href="https://colab.research.google.com/github/Untick/Marketplaces/blob/Kosmylev-Vladislav-branch/3_svod_DataSet_vlad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка

#### Загрузка исходников

**Загрузка библиотек:**

https://drive.google.com/file/d/1-_LKbDsiq0dDbks7_JnBJVV7v2iWjSkd/view?usp=sharing

https://drive.google.com/file/d/1CzIZl5hxOpNsEiWkcJgjXRsAVJz4RbsX/view?usp=sharing  target_tab.zip

In [2]:
import numpy as np                        # Импорт библиотеки numpy
import pandas as pd                       # для анализа табличных массивов данных
from pandas.core.groupby import NamedAgg
import gdown, os, re, random                  # Стандартные библиотеки

In [3]:
!gdown --id '1-_LKbDsiq0dDbks7_JnBJVV7v2iWjSkd'
#!gdown --id '1CzIZl5hxOpNsEiWkcJgjXRsAVJz4RbsX'
!rm -r sample_data
#!rm base_for_analiz.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-_LKbDsiq0dDbks7_JnBJVV7v2iWjSkd
To: /content/base_for_analiz.zip
100% 88.5M/88.5M [00:00<00:00, 93.8MB/s]


In [ ]:
!unzip -o /content/base_for_analiz.zip
# !unzip -o /content/target_tab.zip
# !rm target_tab.zip
!rm base_for_analiz.zip

#### Формируем ДатаФреймы

In [46]:
def travel_dir(folder_path, subdir=True):
    file_dict = {}
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_name = os.path.splitext(file)[0]
            file_dict[file_name] = file_path
        if not subdir:
          return file_dict
    return file_dict


##### Пути по сегментам

In [ ]:
fdir = '/content/base_for_analiz/Ozon/Товары по категориям'
cat_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/Ozon/Performance/Трафареты'
traf_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/Ozon/Performance/ПВП'
pvp_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/Ozon'
ozon_paths = travel_dir(fdir, False)

fdir = '/content/base_for_analiz/MPSTATS/Категория/MPSTATS Категория по дням'
mpcat_day_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/MPSTATS/Категория/MPSTATS по категории (по всем продавцам)'
mpcat_sel_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/MPSTATS/Магазин/MPSTATS 01062022-31052023 Выручка по дням'
mpmag_gros_paths = travel_dir(fdir)

fdir = '/content/base_for_analiz/MPSTATS/Магазин/MPSTATS 01062022-31052023 Товары'
mpmag_tov_paths = travel_dir(fdir)


##### Загрузка исходников в ДатаФреймы

In [ ]:
all_df = {}

In [13]:
# Нормализуем заголовки столбцов
remove_list = ["(при наличии)",                 # список кусков текста для удаления из названий столбцов
               "(кроме Твери)",
               "руб.",
               "(перечеркнутая цена)",
               ",",
               "%",
               "с учетом скидки",
               "на рыночную цену"]
def clean_header(header):

    cleaned_header = header
    cleaned_header = cleaned_header.replace('-', "_")
    cleaned_header = cleaned_header.replace('" ', '"')
    cleaned_header = cleaned_header.replace(' "', '"')

    for item in remove_list:
        cleaned_header = cleaned_header.replace(item, "")

    cleaned_header = cleaned_header.replace('  ', '')
    cleaned_header = cleaned_header.replace('_"', '"')
    cleaned_header = cleaned_header.strip()  # удаление ведущих и концевых пробелов
    cleaned_header = cleaned_header.replace(" ", "_")  # замена оставшихся пробелов символом "_"
    cleaned_header = cleaned_header.replace('_"', '"')

    list_heads = cleaned_header.replace(';', ',')
    list_heads = list_heads.replace('"', '')
    list_heads = list_heads.replace('_"', '"')
    # print(header, " --->  ", list_heads)

    return cleaned_header

def clean_csv(file_path):
    with open(file_path, 'r', newline='') as file:
        lines = file.readlines()
        cleaned_headers = clean_header(lines[0]);  # применяем функцию clean_header к каждому заголовку
        lines[0] = cleaned_headers + '\n'
        print(cleaned_headers)

    with open(file_path, 'w', newline='') as file:
        file.writelines(lines)

    return cleaned_headers


###### Ozon

In [60]:
os.getcwd()
os.chdir('/content/base_for_analiz/Ozon/')

'/content/base_for_analiz/Ozon'

In [67]:
columns = ['Артикул','Ozon_Product_ID','FBO_OZON_SKU_ID','FBS_OZON_SKU_ID','Контент_рейтинг','Текущая_цена']
xls = 'База.csv'
list_heads = clean_csv(xls)


﻿"Артикул";"Ozon_Product_ID";"FBO_OZON_SKU_ID";"FBS_OZON_SKU_ID";"Barcode";"Наименование_товара";"Контент_рейтинг";"Бренд";"Статус_товара";"Видимость_FBO";"Причины_скрытия_FBO";"Видимость_FBS";"Причины_скрытия_FBS";"Дата_создания";"Коммерческая_категория";"Объем_товара_л";"Объемный_вес_кг";"Доступно_на_складе_Ozon_шт";"Вывезти_и_нанести_КИЗшт";"Зарезервировано_шт";"Доступно_на_моих_складах_шт";"Зарезервировано_на_моих_складах_шт";"Текущая_цена";"Цена_до_скидки";"Цена_Premium";"Рыночная_цена";"Актуальная_ссылка";"Размер_НДС"


In [70]:
df = pd.read_csv(xls, sep=';', encoding='utf-8', usecols=columns) #

In [73]:
df.head(3)

,Артикул,Ozon_Product_ID,FBO_OZON_SKU_ID,FBS_OZON_SKU_ID,Контент_рейтинг,Текущая_цена
0,'30273,105606938,298131117,298131122,'15,1200.0
1,'30383,105742204,298264198,298264234,'37.5,1990.0
2,'31284,112525698,303785665,303785701,'30,7900.0


In [21]:
xls = '/content/base_for_analiz/Ozon/Аналитика Акции 01.06.2022-31.05.2023.xlsx'
act = pd.read_excel(xls)   #, encoding='cp1251' если выдаёт ошибку

In [22]:
xls = '/content/base_for_analiz/Ozon/Аналитика Графики 01.06.2022-31.05.2023.xlsx'
graf = pd.read_excel(xls)

In [26]:

xls = '/content/base_for_analiz/Ozon/Товары по категориям/2023-06-05 Автомобильные лампы ксенон комплектующие.xlsx'
pr_kat = pd.read_excel(xls, sheet_name='Шаблон для поставщика', skiprows=[0,2],index_col=[])

In [ ]:
xls = '/content/trafaret/1223100_22.06.2022-22.08.2022.csv'
traf = pd.read_csv(xls, skiprows=[0]) #, skiprows=[0,2], usecols=['column1', 'column2']

In [ ]:
xls = '/content/pvp/1415722_08.08.2022-08.10.2022.csv'
traf = pd.read_csv(xls, skiprows=[0])